# Analyst Price Target
*[Source](https://wrds-web.wharton.upenn.edu/wrds//ds/ibes/ptgdet/index.cfm)*

In [1]:
NAME = '06-02_analysts_price_target'
PROJECT = 'conference-calls-sentiment'
PYTHON_VERSION = '3.7.0'

### Imports  

In [2]:
import os
import re
import numpy as np
import pandas as pd

### Settings

In [3]:
workdir = re.sub("(?<={})[\w\W]*".format(PROJECT), "", os.getcwd())
os.chdir(workdir)

pipeline = os.path.join('2_pipeline', NAME)
if not os.path.exists(pipeline):
    os.makedirs(pipeline)
    for folder in ['out', 'store', 'tmp']:
        os.makedirs(os.path.join(pipeline, folder))

---
# Main Code 

In [ ]:
cols = {'OFTIC': 'ticker', 'TICKER': 'ibes_ticker', 'CNAME': 'coname',
        'ESTIMID': 'brokerage', 'ALYSNAM': 'analyst', 'VALUE': 'value',
        'AMASKCD': 'analyst_id', 'ANNDATS': 'date', 'ANNTIMS': 'time'}

price_target_raw = pd.read_csv(os.path.join('0_data', 'ibes', 'ibes_price-target_2000-2020.csv.gz'), usecols=cols, encoding='latin-1')

price_target_raw.head()

## Data manipulation

In [5]:
def add_previous_valuation(df):
    df[['prev_value', 'prev_date']] = df.groupby(['ticker', 'analyst_id'])[['value', 'date']].shift(1)
    df['days_since_prev'] = df['date'] - df['prev_date']
    df['price_change'] = df['value'] - df['prev_value']
    df['price_sentiment'] = np.where(df['price_change'] > 0, 1, -1)
    df['price_sentiment'] = np.where(df['price_change'] == 0, 0, df['price_sentiment'])
    return df

In [ ]:
price_target = (price_target_raw
                .copy()
                .dropna()
                .rename(columns=cols)
                .assign(
                    date=lambda x: pd.to_datetime(x['date'], format=r'%Y%m%d'),
                    analyst=lambda x: x['analyst'].apply(lambda a: ' '.join(a.split()))  # Remove extra whitespaces between analyst names
                    )
                .sort_values(['ticker', 'analyst_id', 'date', 'time'])
                .drop_duplicates(['ticker', 'analyst_id', 'date'])
                .pipe(add_previous_valuation)
                .dropna()
                .reset_index(drop=True)
                .filter(['ticker', 'coname', 'brokerage', 'brokarage_id',
                         'analyst', 'analyst_id', 'date', 'days_since_prev',
                         'prev_value', 'value', 'price_change', 'price_sentiment']))

price_target.head()

## Save `analyst_id`-`analyst` mapping

In [7]:
analyst_mapping = (price_target.copy()
                   .filter(['analyst', 'analyst_id', 'ticker'])
                   .drop_duplicates(['analyst_id', 'ticker'])
                   .reset_index(drop=True))
                   
analyst_mapping.to_feather(os.path.join(pipeline, 'store', 'analyst_mapping.feather'))

## Match analyst's rating with conference calls transcript

In [8]:
cc = pd.read_feather(os.path.join('2_pipeline', '02-02_conference_calls_preprocess', 'out', 'cc_transcripts.feather'))
cc = (cc.filter(['gvkey', 'ticker', 'event_date', 'speaker_role', 'speaker_name', 'speaker_firm'])
        .query("speaker_role == 'Analyst'")
        .drop_duplicates()
        .reset_index(drop=True))
cc.head()

In [ ]:
# Merge to transcripts
price_target_cc = (price_target.merge(cc,
                                      left_on=['ticker', 'analyst'],
                                      right_on=['ticker', 'speaker_name'])
                               .assign(days_between=lambda x: x['date'] - x['event_date'])
                               .query('0 <= days_between.dt.days <= 20')
                               .sort_values(['gvkey', 'analyst', 'brokerage', 'date', 'days_between'])
                               .drop_duplicates(['gvkey', 'analyst', 'brokerage', 'date'], keep='first'))

len(price_target_cc)

## Recommendations sentiment by analysts

In [ ]:
price_target_sentiment_analyst = (price_target_cc
                                  .copy()
                                  .reset_index(drop=True)
                                  .filter(['gvkey', 'event_date', 'analyst', 'price_change', 'price_sentiment']))
price_target_sentiment_analyst

In [11]:
price_target_sentiment_analyst.to_feather(os.path.join(pipeline, 'out', 'price_level_analyst.feather'))